Extracting the features is only the first part. Before feeding the features into the model pipeline, it is necessary to create the feature matrix.

In [ ]:
%pylab
%matplotlib inline
from __future__ import division
import matplotlib.pyplot as plt
import os
import tables
import json
import music.features.DesignMatrix as DM
import numpy as np

print "done importing"

In [ ]:
data_dir = os.path.abspath(os.path.join(os.getcwd(), '..', 'tests', 'data'))
trname = 'sess_1_block_0_AN.report' # sample report file

### Import the sample extracted features

In [ ]:
all_features = {}
f = tables.open_file('sampleFeatures.hdf')

In [ ]:
toOpen = []
for array in f.walk_nodes('/', 'Array'):
    arrsplit = str(array).split(' ')
    toOpen.append(arrsplit[0][1:])

In [ ]:
print toOpen

In [ ]:
all_features = {}

all_features['CQT'] = f.root.CQT.read()
all_features['FP'] = f.root.FP.read()
all_features['MELSPECT'] = f.root.MELSPECT.read()
all_features['MFCC'] = f.root.MFCC.read()
all_features['RMS'] = f.root.RMS.read()
all_features['S'] = f.root.S.read()
all_features['ZCR'] = f.root.ZCR.read()
all_features['chroma'] = f.root.chroma.read()
all_features['keyTimeSeries'] = f.root.keyTimeSeries.read()
all_features['modeTimeSeries'] = f.root.modeTimeSeries.read()

f.close()

In [ ]:
with open('featureParams.json',) as data_file:
    extractionParameters = json.load(data_file)

### Creating a feature matrix

You can use a DesignMatrix object to construct a feature matrix to be used for model fitting. If you have a TR report file for the data session of interest, you should pass it in as a parameter for the DesignMatrix constructer. The TR report will be used for downsampling extracted features to correctly correspond to the number of TRs in that session.

Later in the demo, you'll see how to create a design matrix when there is no TR file. In the case of no TR report, merely leave the parameter blank and it will default to a value of None.

In [ ]:
TR=2.0045
sr=extractionParameters['RMS']['sr']

In [ ]:
mat_maker = DM.DesignMatrix(all_features, trfile=os.path.join(data_dir, trname), TR=TR, sr=sr)

#### Create a feature matrix without downsampling. 

To do this, all features already need to be the same size.

In [ ]:
featsets = [
            'RMS',
            'ZCR',
            'MFCC',
            'MELSPECT',
            'S',
            ]

The matrix constructor has a built-in safety. You need to pass in as a parameter the size of the features you are combining. If any of the features isn't that size it will be skipped when building the feature matrix.

In [ ]:
## the length of the features to be used
time_len = mat_maker.returnFeature('RMS').shape[1]
## make the feature matrix
feat_mat = mat_maker.buildMatrix(featsets,time_len)

In [ ]:
print feat_mat.shape

The dimensions of the feature matrix is time x features, where each feature is some time x n matrix (n being the number of dimensions that make up that feature, i.e. RMS is time x 1, whereas S is time x 1025).

In [ ]:
rms = feat_mat[:,0]

In [ ]:
plt.figure()
plt.plot(rms)
plt.title('RMS')
plt.xlabel('time')
plt.show()

The first column of the feature matrix matches the extracted root mean sqaure feature from before.

#### Downsampling a feature matrix

After constructing the feature matrix, you can then downsample it to be the number of TRs. The default method of downsampling is using lanzcos interpolation. You can opt to apply a nonlinearity to the extracted feature values before downsampling, otherwise no nonlinearity will be applied and the values will be used as they are. See documentation of DesignMatrix.py for more details.

In [ ]:
# parameters necessary for downsampling
endTrim = False

# can either set number if remember it, or can access via the extraction parameters used
n_fft = 2048
# n_fft = extractionParameters['RMS']['n_fft']

# length of the audio file from which the features were extracted
audiolen = extractionParameters['RMS']['audio_len_samples']

The endTrim parameter was added to account for the fact that some studies have blank TRs at the beginning and the end of the run (as in the scanning sequence is running for a few seconds at the beginning and the end without any stimulus presentation). The number of TRs to snip is also a parameters, with a default value of 5 [TRs].

These parameters are only relevant for data that have TR reports. See documentation of DesignMatrix.downsampleMatrix, DesignMatrix.calculateOldtimes, DesignMatrix.calculateTRtimes, and DesignMatrix.interpolateDownsample for full details. 

In [ ]:
downsamp_featmat = mat_maker.downsampleMatrix(time_len, audiolen=audiolen, endTrim=endTrim, n_fft=n_fft)

In [ ]:
print downsamp_featmat.shape

In [ ]:
rms_downsamp = downsamp_featmat[:,0]

In [ ]:
plt.figure()
plt.plot(rms_downsamp)
plt.title('Downsampled RMS')
plt.xlabel('Time')
plt.show()

As a sanity check, the first column corresponds to the downsampled RMS feature, which has maintained the same general shape (albeit the values have changed).

#### Downsampling just a feature, not a full matrix.

The above downsampled a full feature matrix. You can also downsample individual features.

In [ ]:
FP = all_features['FP']

# original length of the feature before downsampling
oldlen = FP.shape[1]

# the new length of the downsampled feature
newlen = len(mat_maker.calculateTRtimes(endTrim=endTrim))

downsamp_FP = mat_maker.downsampleFeature(FP, oldlen, method='resample', newlen=newlen)

With the current implementation of the feature extraction module, downsampling the fluctuation patterns feature requires using a resampling method (i.e, sinc interpolation instead of using lanczos resampling). To do this, you'll need to pass in the the original length of the downsampled features (in time domain) and the new length of the features, after downsampling.

If you don't have a TR report file, an example of how to downsample will be shown later in the tutorial.

In [ ]:
dims = FP.shape
dims_downsamp = downsamp_FP.shape

In [ ]:
plt.figure()
plt.imshow(np.flipud(FP), aspect='auto', cmap='gray')
plt.xlabel('Time segment number')
plt.ylabel('Amplitude modulation per mel frequency band (Hz)')
plt.yticks(np.r_[0:dims[0]:dims[0]/12][::-1], np.arange(1,13))
plt.title('Fluctuation Patterns')
plt.show()

In [ ]:
plt.figure()
plt.imshow(np.flipud(downsamp_FP), aspect='auto', cmap='gray')
plt.xlabel('Time segment number')
plt.ylabel('Amplitude modulation per mel frequency band (Hz)')
plt.yticks(np.r_[0:dims_downsamp[0]:dims_downsamp[0]/12][::-1], np.arange(1,13))
plt.title('Fluctuation Patterns (downsampled)')
plt.show()

### Combining different features of different lengths

If you wish to construct a feature matrix that consists of several different extracted features that are of different lengths, then you need to downsample the features individually and stack them together to form a full feature matrix. This will happen if you wish to combine short duration features (spectral features) with long duration features (rhythmic/tonal features).

In [ ]:
TR=2.0045
sr=extractionParameters['RMS']['sr']

In [ ]:
mat_maker = DM.DesignMatrix(all_features, trfile=os.path.join(data_dir, trname), TR=TR, sr=sr)

First combine all the short duration features into a matrix and downsample the matrix as before. These all were extracted using the same windowing and are therefore the same length in the time domain.

In [ ]:
featsets = [
            'RMS',
            'ZCR',
            'MFCC',
            'MELSPECT',
            'S',
            ]

## the length of the features to be used
time_len = mat_maker.returnFeature('RMS').shape[1]
## make the feature matrix
feat_mat = mat_maker.buildMatrix(featsets,time_len)

In [ ]:
# parameters necessary for downsampling
endTrim = False

# can either set number if remember it, or can access via the extraction parameters used
n_fft = 2048
# n_fft = extractionParameters['RMS']['n_fft']

# length of the audio file from which the features were extracted
audiolen = extractionParameters['RMS']['audio_len_samples']

In [ ]:
downsamp_featmat = mat_maker.downsampleMatrix(time_len, audiolen=audiolen, endTrim=endTrim, n_fft=n_fft)

Now we individually downsample the other features of interest which either uses methods other than lanczos resampling or are of a different length in the time domain.

The current implementation of the feature extraction library is set up such that certain features are downsample in specific ways:
+ chroma & short duration features: lanzcos resampling ('inter')
+ tonality: downsampling by only taking every nth sample, where n is the downsample factor ('man')
+ FP: downsampling using sinc interpolation and resampling ('resample')
    - this can be changed to lanzcos interpolation, but requires implementing a function that will accurately compute the time points of the pre-downsampled feature
+ CQT: downsampling using lanczos resampling, but modified to work with the CQT feature ('interCQT')

In [ ]:
downf = []

# load specific features to downsample prior to use -- longer duration features
C = mat_maker.returnFeature('chroma')
K = mat_maker.returnFeature('keyTimeSeries')
M = mat_maker.returnFeature('modeTimeSeries')
FP = mat_maker.returnFeature('FP')
CQT = mat_maker.returnFeature('CQT')

In [ ]:
# new length after downsampling
newlen = len(mat_maker.calculateTRtimes(endTrim=endTrim))

# the length of current time series before downsampling
# oldlen same value for C, K, M with the extraction parameters used for the demos
oldlen = K.shape[1]
assert K.shape[1] == C.shape[1]
assert M.shape[1] == C.shape[1]

# hop length used for the long duration features
long_hop = extractionParameters['chroma']['hop_length']
long_n_fft = extractionParameters['chroma']['n_fft']

# chromagram (uses lanczos resampling)
downf.append(mat_maker.downsampleFeature(C, oldlen, audiolen=audiolen, method='inter', endTrim=endTrim,
                                         n_fft=long_n_fft, hop_length=long_hop))

# tonality (uses manual downsampling -- choose every nth sample, where n is the downsample factor)
downf.append(mat_maker.downsampleFeature(K, oldlen, method='man', newlen=newlen))
downf.append(mat_maker.downsampleFeature(M, oldlen, method='man', newlen=newlen))

# fluctuation patterns (pre-downsampling length for fluctuation patters )
oldlen = FP.shape[1]

# fluctuation patterns (uses sinc interpolation to downsample)
downf.append(mat_maker.downsampleFeature(FP, oldlen, method='resample', newlen=newlen))

# CQT extraction parameters needed for feature downsampling
cqt_hop = extractionParameters['CQT']['cqt_hop']
cqt_seconds = extractionParameters['CQT']['frame_seconds']
oldlen = CQT.shape[1]

# constant q transform
downf.append(mat_maker.downsampleFeature(CQT, oldlen, method='interCQT', endTrim=endTrim, cqt_hop=cqt_hop,
                                         seconds=cqt_seconds, sr=sr))

In [ ]:
# this variable is to make sure all features downsampled to same length
time_len = downf[0].shape[1]

# build long duration matrix
tmpL = mat_maker.buildMatrix(downf, time_len, retrieve=False)

# stack short and long duration features together and store the matrix in the DesignMatrix instance
full_feat_mat = np.hstack((downsamp_featmat,tmpL))
mat_maker.assignMatrix(full_feat_mat)

In [ ]:
featsets += [
            'chroma',
            'keyTimeSeries',
            'modeTimeSeries',
            'FP',
            'CQT'
            ]

# store the features used for this particular design matrix as an attribute of the DesignMatrix instance
mat_maker.assignFeatsets(featsets)

In [ ]:
full_feat_mat.shape

Now we've downsampled the full feature matrix and it consists of both short- and long- duration features.

In [ ]:
rms_downsamp = full_feat_mat[:,0]

plt.figure()
plt.plot(rms_downsamp)
plt.title('RMS downsampled')
plt.xlabel('Time')
plt.show()

As a sanity check, here is the downsampled RMS feature plotted, matches the original.

### Downsampling without a TR report

Sometimes you may need to downsample your features but don't have an accompanying TR report file for the dataset.

##### Downsampling an individual feature

In [ ]:
TR=2.0045
sr=extractionParameters['RMS']['sr']
endTrim = False

In [ ]:
mat_maker = DM.DesignMatrix(all_features, TR=TR, sr=sr)

In [ ]:
# necessary parameter, new length of the downsampled feature
newLen = 216

CQT = mat_maker.returnFeature('CQT')
oldlen = CQT.shape[1]
cqt_hop = extractionParameters['CQT']['cqt_hop']
cqt_seconds = extractionParameters['CQT']['frame_seconds']

All the other parameters are the same as before, but there is one new parameter required when there is no TR report. You need to pass in the desired length (of the downsampled feature) into DesignMatrix.downsampleMatrix() or DesignMatrix.downsampleFeature().

In [ ]:
downsamp_CQT = mat_maker.downsampleFeature(CQT, oldlen, method='interCQT', endTrim=endTrim, newLen=newLen,
                            cqt_hop=cqt_hop, seconds=cqt_seconds, sr=sr)

In [ ]:
downsamp_CQT.shape

In [ ]:
plt.figure()
plt.imshow(CQT, aspect='auto')
plt.title('Constant Q Transform')
plt.show()

plt.figure()
plt.imshow(downsamp_CQT, aspect='auto')
plt.title('Constant Q Transform, downsampled')
plt.show()

##### Downsampling a full matrix

In [ ]:
featsets = [
            'RMS',
            'ZCR',
            'MFCC',
            'MELSPECT',
            'S',
            ]

In [ ]:
## the length of the features to be used
time_len = mat_maker.returnFeature('RMS').shape[1]
## make the feature matrix
feat_mat = mat_maker.buildMatrix(featsets,time_len)

In [ ]:
# parameters necessary for downsampling
endTrim = False

newLen = 216

# can either set number if remember it, or can access via the extraction parameters used
n_fft = 2048
# n_fft = extractionParameters['RMS']['n_fft']

# length of the audio file from which the features were extracted
audiolen = extractionParameters['RMS']['audio_len_samples']

In [ ]:
downsamp_featmat = mat_maker.downsampleMatrix(time_len, audiolen=audiolen, endTrim=False, newLen=newLen, n_fft=n_fft)

In [ ]:
rms = feat_mat[:,0]
rms_downsamp = downsamp_featmat[:,0]

In [ ]:
plt.figure()
plt.plot(rms)
plt.title('RMS')
plt.xlabel('time')
plt.show()

plt.figure()
plt.plot(rms_downsamp)
plt.title('RMS, downsampled')
plt.xlabel('time')
plt.show()

### Saving the design matrix

You can also save the resulting feature matrix.

In [ ]:
full_feat_mat = mat_maker.saveMatrix()
featsets = mat_maker.returnFeatsets()

In [ ]:
toSave = dict(DesignMatrix=full_feat_mat, DesignFeats=featsets)

In [ ]:
# hf = tables.open_file('DesignMatrix.hdf', mode='w', title='safe_file')
# for vname, var in toSave.items():
#     hf.create_array('/', vname, var)
# hf.close()

### Note

The order you pass in parameter values will make a difference. Look at source code and examples in tutorials for proper usage.